In [102]:
from lyricsgenius import Genius
from collections import Counter
import re
from nltk.tokenize import TreebankWordTokenizer
import string
import pandas as pd
import requests
from tqdm.notebook import tqdm

tokenizer = TreebankWordTokenizer()
punct = set(string.punctuation)
punct.update({"''", "``", ""})
genius = Genius('bVEbboB9VeToZE48RaiJwrnAGLz8VbrIdlqnVU70pzJXs_T4Yg6pdPpJrTQDK46p')
genius.verbose = False

In [103]:
path = r'C:\Users\chris\Documents\GitHub\cs4300sp2021-rad338-jsh328-rpp62-cmc447\sample_data'
df = pd.read_csv(path + r"\sample_scraped_data200.csv")

In [113]:
def match(a, b):
    return (a in b) or (b in a)

def get_lyrics(song_name, song_artist):
    s = genius.search_song(song_name, song_artist, get_full_info = False)
    if s:
        s = s.to_dict()
        query_sn, query_sa = song_name.lower(), song_artist.lower()
        returned_sn, returned_sa = s['artist'].lower(), s['title'].lower()

        if match(query_sn, returned_sn) and match(query_sa, returned_sa):
            return s['lyrics']

In [119]:
def get_all_lyrics(df):
    """
    df: DataFrame; uri, name, and artist must be columns in the dataframe
    
    returns a dictionary where key = song uri and value = Counter of tokenized lyrics
    """
    cnt_dict = dict() #dict(uri: Counter(lyric tokens))
    pbar = tqdm(total=len(df))
    for _, row in df.iterrows():
        uri, name, artist = row[['uri', 'name', 'artist']]
        if "-" in name:
            name = name[:name.find("-")].strip()
        lyrics = get_lyrics(name, artist)
        if lyrics:
            lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
            tokens = [t for t in tokenizer.tokenize(lyrics) if t not in punct]
            cnt = Counter(tokens)
            cnt_dict[uri] = cnt
        pbar.update(1)
    return cnt_dict

In [120]:
get_all_lyrics(df)

  0%|          | 0/1066 [00:00<?, ?it/s]

KeyboardInterrupt: 